In [8]:
import pandas as pd
import mlflow
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import logging

In [9]:
# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class SentimentValidator:
    def __init__(self, parquet_path):
        self.vectorizer = None
        self.label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
        self.inverse_mapping = {v: k for k, v in self.label_mapping.items()}
        self.parquet_path = parquet_path

    def load_data(self, parquet_path):
        """Carrega e transforma os dados de validação"""
        try:
            df = pd.read_parquet(parquet_path)
            
            # Verificação de colunas essenciais
            if not all(col in df.columns for col in ['comment_cleaned', 'sentiment']):
                raise ValueError("Colunas 'comment_cleaned' ou 'sentiment' não encontradas")
            
            # Filtragem e limpeza
            df = df[df['sentiment'].isin(self.label_mapping.keys())]
            df = df.dropna(subset=['comment_cleaned', 'sentiment'])
            
            if len(df) == 0:
                raise ValueError("Nenhum dado válido após filtragem")
                
            return df['comment_cleaned'].values, df['sentiment'].values
        
        except Exception as e:
            logger.error(f"Erro ao carregar dados: {str(e)}")
            raise

    def load_model_and_components(self, model_name):
        """Carrega o modelo e extrai o vetorizador corretamente"""
        try:
            model_uri = f"models:/sentiment_{model_name}/latest"
            
            sklearn_model = mlflow.sklearn.load_model(model_uri)
            return sklearn_model
        
        except Exception as e:
            model_uri = f"models:/{model_name}/latest"
            
            sklearn_model = mlflow.sklearn.load_model(model_uri)
            return sklearn_model

    def transform_data(self, X):
        """Transforma os dados conforme o pipeline de treinamento"""
        if self.vectorizer is None:
            logger.warning("Vetorizador não encontrado, criando novo como fallback")
            self.vectorizer = TfidfVectorizer(max_features=5000)
            
            # Apenas fit se for um novo vetorizador (evitar data leakage)
            self.vectorizer.fit(X)
        
        return self.vectorizer.transform(X)

    def validate(self, model_name="randomforest"):
        """Executa a validação completa"""
        try:
            with mlflow.start_run(run_name=f"Validation_{model_name}"):
                # 1. Carregar dados
                X_val, y_val_true = self.load_data(self.parquet_path)
                logger.info(f"Dados carregados: {len(X_val)} amostras")
                
                # 2. Carregar modelo e componentes
                model = self.load_model_and_components(model_name)
                
                # 3. Fazer previsões diretamente (o modelo já inclui o pipeline completo)
                y_val_pred = model.predict(X_val)
                
                # 4. Converter labels numéricos para texto se necessário
                if all(isinstance(x, (int, float, np.integer)) for x in y_val_pred):
                    y_val_pred = [self.inverse_mapping.get(int(x), 'Neutral') for x in y_val_pred]
                
                # 5. Calcular métricas
                self._log_metrics(y_val_true, y_val_pred, model_name)
                
                return True
                
        except Exception as e:
            logger.error(f"Falha na validação: {str(e)}")
            return False

    def _log_metrics(self, y_true, y_pred, model_name):
        """Calcula e registra métricas no MLflow"""
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        report = classification_report(y_true, y_pred, output_dict=True)
        
        # Log básico
        mlflow.log_metrics({
            "val_accuracy": accuracy,
            "val_f1_weighted": f1
        })
        
        # Log por classe
        for cls in ['Negative', 'Neutral', 'Positive']:
            if cls in report:
                mlflow.log_metrics({
                    f"val_precision_{cls.lower()}": report[cls]['precision'],
                    f"val_recall_{cls.lower()}": report[cls]['recall'],
                    f"val_f1_{cls.lower()}": report[cls]['f1-score'],
                    f"val_support_{cls.lower()}": report[cls]['support']
                })
        
        # Matriz de confusão
        self._plot_confusion_matrix(y_true, y_pred, model_name)
        
        logger.info(f"\nModelo: {model_name}")
        logger.info(f"Acurácia: {accuracy:.4f}")
        logger.info(f"F1-Score: {f1:.4f}")
        logger.info("\nRelatório de Classificação:")
        logger.info(classification_report(y_true, y_pred))

    def _plot_confusion_matrix(self, y_true, y_pred, model_name):
        """Gera e salva a matriz de confusão"""
        cm = confusion_matrix(y_true, y_pred, labels=['Negative', 'Neutral', 'Positive'])
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Negative', 'Neutral', 'Positive'],
                    yticklabels=['Negative', 'Neutral', 'Positive'])
        plt.title(f'Matriz de Confusão - {model_name}')
        plt.ylabel('Verdadeiro')
        plt.xlabel('Previsto')
        
        cm_path = f"confusion_matrix_{model_name}.png"
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path)
        plt.close()

In [10]:
# Configuração
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("Validation_Trim_FixNeg_Restaurant_Sentiment")

parquet_path = "../data\dataset_valid_with_sentiment_fix_negative_trimmed_similarity.parquet"

validator = SentimentValidator(parquet_path=parquet_path)

# Lista de modelos para validar
models_to_validate = ['randomforest', 'logisticregression', 'gs_logisticregression']

for model_name in models_to_validate:
    logger.info(f"\nIniciando validação para {model_name}...")
    success = validator.validate(model_name)
    
    if success:
        logger.info(f"Validação de {model_name} concluída com sucesso!")
    else:
        logger.info(f"Validação de {model_name} falhou.")

2025/05/14 15:30:31 INFO mlflow.tracking.fluent: Experiment with name 'Validation_Trim_FixNeg_Restaurant_Sentiment' does not exist. Creating a new experiment.
INFO:__main__:
Iniciando validação para randomforest...
INFO:__main__:Dados carregados: 195 amostras


INFO:__main__:
Modelo: randomforest
INFO:__main__:Acurácia: 0.6923
INFO:__main__:F1-Score: 0.6370
INFO:__main__:
Relatório de Classificação:
INFO:__main__:              precision    recall  f1-score   support

    Negative       0.70      0.42      0.53        50
     Neutral       0.33      0.04      0.07        25
    Positive       0.70      0.94      0.80       120

    accuracy                           0.69       195
   macro avg       0.58      0.47      0.47       195
weighted avg       0.65      0.69      0.64       195

INFO:__main__:Validação de randomforest concluída com sucesso!
INFO:__main__:
Iniciando validação para logisticregression...
INFO:__main__:Dados carregados: 195 amostras


🏃 View run Validation_randomforest at: http://127.0.0.1:5000/#/experiments/181704948755466937/runs/69bc4a99f01f4651abf98e8b4913c69c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/181704948755466937


INFO:__main__:
Modelo: logisticregression
INFO:__main__:Acurácia: 0.6821
INFO:__main__:F1-Score: 0.5995
INFO:__main__:
Relatório de Classificação:
INFO:__main__:              precision    recall  f1-score   support

    Negative       0.92      0.24      0.38        50
     Neutral       0.50      0.04      0.07        25
    Positive       0.67      1.00      0.80       120

    accuracy                           0.68       195
   macro avg       0.70      0.43      0.42       195
weighted avg       0.71      0.68      0.60       195

INFO:__main__:Validação de logisticregression concluída com sucesso!
INFO:__main__:
Iniciando validação para gs_logisticregression...
INFO:__main__:Dados carregados: 195 amostras


🏃 View run Validation_logisticregression at: http://127.0.0.1:5000/#/experiments/181704948755466937/runs/f4a62e7094b548788456a2ddaa89ee72
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/181704948755466937


INFO:__main__:
Modelo: gs_logisticregression
INFO:__main__:Acurácia: 0.6821
INFO:__main__:F1-Score: 0.6424
INFO:__main__:
Relatório de Classificação:
INFO:__main__:              precision    recall  f1-score   support

    Negative       0.67      0.44      0.53        50
     Neutral       0.22      0.08      0.12        25
    Positive       0.71      0.91      0.80       120

    accuracy                           0.68       195
   macro avg       0.53      0.48      0.48       195
weighted avg       0.64      0.68      0.64       195

INFO:__main__:Validação de gs_logisticregression concluída com sucesso!


🏃 View run Validation_gs_logisticregression at: http://127.0.0.1:5000/#/experiments/181704948755466937/runs/50b937cf550342f9ac5e1bb4d77853d7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/181704948755466937
